In [49]:
import pandas as pd
import numpy as np

In [63]:
import tensorflow as tf

import keras.models 

from keras.models import Model
from keras.layers import Dense

import keras_metrics

Using TensorFlow backend.


In [35]:
rf_people_preds = pd.read_csv("../rf_validation_prob.csv")[["1"]]
nn_people_preds = pd.read_csv("../val_predictions_people_prob_nn.csv")
labels = pd.read_csv("../Archive/people_validate.csv")[["suspicious"]]

In [21]:
rf_people_preds.head()

,1
0,0.723247
1,0.000997
2,0.161828
3,0.050795
4,0.183324


In [22]:
nn_people_preds.head()

,predictions
0,0.564710
1,0.019289
2,0.030049
3,0.036152
4,0.189456


In [42]:
all_preds = pd.concat((nn_people_preds, rf_people_preds), axis=1)
all_preds.head()

,predictions,1
0,0.564710,0.723247
1,0.019289,0.000997
2,0.030049,0.161828
3,0.036152,0.050795
4,0.189456,0.183324


In [38]:
preds = all_preds.values

In [39]:
labels = labels.values

In [41]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [44]:
recall_rf = recall_score(list(map(lambda x: 0 if x < 0.5 else 1, preds[:,1])), labels)
recall_rf

0.08227848101265822

In [45]:
precision_rf = precision_score(list(map(lambda x: 0 if x < 0.5 else 1, preds[:,1])), labels)
precision_rf

0.5348837209302325

In [46]:
recall_nn = recall_score(list(map(lambda x: 0 if x < 0.5 else 1, preds[:,0])), labels)
recall_nn

0.06799026034837985

In [47]:
precision_nn = precision_score(list(map(lambda x: 0 if x < 0.5 else 1, preds[:,0])), labels)
precision_nn

0.6493738819320215

In [50]:
avg_preds = np.mean(preds, axis=1)
avg_preds.shape

(160036,)

In [51]:
recall_avg = recall_score(list(map(lambda x: 0 if x < 0.5 else 1, avg_preds)), labels)
recall_avg

0.07790093791178979

In [52]:
precision_nn = precision_score(list(map(lambda x: 0 if x < 0.5 else 1, avg_preds)), labels)
precision_nn

0.5992844364937389

In [53]:
def cost(preds, labels, tresh):
    investigation_cost = sum(preds > tresh) * 1000
    fine_cost = sum(labels[preds < tresh]) * 60000
    return investigation_cost + fine_cost

In [58]:
def best(preds, labels):
    min_cost = -1
    for i in np.linspace(0, 1, 21):
        c = cost(preds, labels, i)
        if min_cost == -1:
            min_cost = c
            
        if c <= min_cost:
            min_cost = c
            min_cost_tresh = i
            
    print("COST: " + str(min_cost) + " for tresh. " + str(min_cost_tresh))

In [59]:
best(preds[:,1],labels)

COST: [55471000] for tresh. 0.35000000000000003


In [60]:
best(preds[:,0],labels)

COST: [50809000] for tresh. 0.45


In [61]:
best(avg_preds,labels)

COST: [50961000] for tresh. 0.4


In [69]:
encoding_dim = preds.shape[-1]
model_input = keras.models.Input(shape=(encoding_dim,))
x = Dense(8, activation='tanh')(model_input)
x = keras.layers.Dropout(0.1)(x)
#x = Dense(8, activation='tanh')(x)
#x = keras.layers.Dropout(0.1)(x)
out = Dense(2, activation='sigmoid')(x)

model = Model(model_input, out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras_metrics.precision(), keras_metrics.recall()])

In [70]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 24        
_________________________________________________________________
dropout_5 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 18        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.fit(preds, tf.keras.utils.to_categorical(labels, 2), epochs=10, batch_size=128, shuffle=True,
                verbose=1, class_weight={0:1, 1:60})

Epoch 1/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.7366 - precision: 0.0592 - recall: 0.6738
Epoch 2/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.6879 - precision: 0.0568 - recall: 0.6947
Epoch 3/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.6885 - precision: 0.0573 - recall: 0.6935
Epoch 4/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.6871 - precision: 0.0572 - recall: 0.7007
Epoch 5/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.6868 - precision: 0.0570 - recall: 0.6947
Epoch 6/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.6883 - precision: 0.0575 - recall: 0.6881
Epoch 7/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.6873 - precision: 0.0569 - recall: 0.6977
Epoch 8/10
160036/160036 [==============================] - 2s 14us/step - loss: 0.6865 - precision: 0.0572 - recall: 0.6941


In [77]:
best(list(map(lambda x: 0 if x < 0.5 else 1, model.predict(preds)[:,1])),labels)

COST: 21011000 for tresh. 0.0


In [90]:
nn_companies_preds = pd.read_csv("../test_predictions_companies_prob_nn.csv")
nn_people_preds = pd.read_csv("../test_predictions_people_prob_nn.csv")

In [91]:
len(nn_companies_preds) + len(nn_people_preds)

1000000

In [92]:
rf_people_preds = pd.read_csv("../test_prob.csv")
rf_companies_preds = pd.read_csv("../test_prob_companies.csv")

In [94]:
nn_preds = pd.concat((nn_companies_preds, nn_people_preds), axis=0).drop(["customer"], axis=1)
len(nn_preds)

1000000

In [97]:
rf_preds = pd.concat((rf_companies_preds, rf_people_preds), axis=0).drop(["0","Unnamed: 0"], axis=1)
len(rf_preds)

1000000

In [98]:
nn_preds.head()

,predictions
0,0.067030
1,0.307272
2,0.150903
3,0.236071
4,0.101547


In [99]:
rf_preds.head()

,1
0,0.046149
1,0.059465
2,0.089963
3,0.043946
4,0.013337


In [100]:
all_test_preds = pd.concat((nn_preds, rf_preds), axis=1)
all_test_preds.head()

,predictions,1
0,0.067030,0.046149
1,0.307272,0.059465
2,0.150903,0.089963
3,0.236071,0.043946
4,0.101547,0.013337


In [107]:
test_predictions = model.predict(all_test_preds.values)[:,1]

In [112]:
sum(list(map(lambda x: 0 if x < 0.5 else 1, test_predictions)))

138416

In [113]:
submission = pd.concat((nn_companies_preds, nn_people_preds), axis=0)[["customer"]]

In [115]:
submission["predictions"] = test_predictions
submission.head()

,customer,predictions
0,91000001,0.096118
1,91000009,0.265327
2,91000010,0.144361
3,91000020,0.196973
4,91000022,0.107676


In [134]:
submission[submission["predictions"] > 0.58][["customer"]].to_csv("../submission.csv", index=False)

In [140]:
cust_ensemble = set(pd.read_csv("../submission.csv")["customer"].values)
cust_ensemble

{91226113,
 91226114,
 91488257,
 91226123,
 91750417,
 91488275,
 91750423,
 91488281,
 91750431,
 91226149,
 91488296,
 91488299,
 91750443,
 91226157,
 91750444,
 91750446,
 91226165,
 91488313,
 91488315,
 91488318,
 91750463,
 91226177,
 91488322,
 91488325,
 91488326,
 91488328,
 91488329,
 91488333,
 91488341,
 91750486,
 91488345,
 91488351,
 91750501,
 91750505,
 91226219,
 91226228,
 91750516,
 91750519,
 91488376,
 91488380,
 91226237,
 91750526,
 91488384,
 91750536,
 91750538,
 91226250,
 91750545,
 91226260,
 91750549,
 91488408,
 91750553,
 91750555,
 91226268,
 91488416,
 91750562,
 91750566,
 91750570,
 91750572,
 91750574,
 91750575,
 91226288,
 91750581,
 91750586,
 91750593,
 91226308,
 91750603,
 91226316,
 91226323,
 91750613,
 91226325,
 91750614,
 91226330,
 91226336,
 91750625,
 91488484,
 91488485,
 91226344,
 91226347,
 91750636,
 91488493,
 91488495,
 91226356,
 91750649,
 91226363,
 91750656,
 91226373,
 91488520,
 91226377,
 91488522,
 91226381,
 91226383,

In [141]:
cust_nn = set(pd.read_csv("../all.csv")["customer"].values)
cust_nn

{91226113,
 91226114,
 91488257,
 91226123,
 91750417,
 91488275,
 91750423,
 91488281,
 91750431,
 91226149,
 91488296,
 91488299,
 91750443,
 91226157,
 91750444,
 91750446,
 91226165,
 91488313,
 91488315,
 91488318,
 91750463,
 91226177,
 91488322,
 91488325,
 91488326,
 91488328,
 91488329,
 91488333,
 91488341,
 91750486,
 91488345,
 91488351,
 91750501,
 91750505,
 91750516,
 91750519,
 91488376,
 91488380,
 91226237,
 91750526,
 91488384,
 91750536,
 91226250,
 91750538,
 91750545,
 91226260,
 91750549,
 91488408,
 91750553,
 91226268,
 91488416,
 91750566,
 91750570,
 91750572,
 91750574,
 91226288,
 91750581,
 91750585,
 91750586,
 91750593,
 91226308,
 91750603,
 91226316,
 91226323,
 91750613,
 91750614,
 91226330,
 91226336,
 91750625,
 91488484,
 91488485,
 91226344,
 91226347,
 91750636,
 91488495,
 91226356,
 91750649,
 91226363,
 91750656,
 91226373,
 91488520,
 91226377,
 91488522,
 91226383,
 91488531,
 91750678,
 91750679,
 91750684,
 91226399,
 91488544,
 91750688,

In [142]:
len(cust_ensemble)

108611

In [138]:
len(cust_nn)

105868

In [139]:
len(cust_nn - cust_ensemble)

3230